# Download and convert to Zarr
This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Zarr Database (based on the zcollection package) for future use.
Zarr (and the way we partitionned data with zcollection) is very efficient for computation. However, it is not (yet) compatible with QGIS compared to Geopackage.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
import pixcdust
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon
gdf_geom = gpd.read_file("../data/aoi.gpkg")

dates = (
    datetime(2024,8,1),
    datetime(2024,8,15),
)

## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a Zarr (zcollection) database.
This Zarr partionned format is very efficient for time analysis, but is not currently accessible in GIS softwares such as QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.zarr import Nc2ZarrConverter
from glob import glob

In [5]:
pixc = Nc2ZarrConverter(
            glob(pixcdownloader.path_download+'/*/*nc'),
            '/tmp/pixc_zarr',
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

database has been succesfully created, we can remove the raw files

In [6]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
previous steps are not necessary

Now we can open this database in a xarray, or dataframe, or GeoDataFrame

In [7]:
from pixcdust.readers.zarr import ZarrReader
import datetime

pixc_read = ZarrReader(
    "/tmp/pixc_zarr"
)
pixc_read.read((datetime.datetime(2024,8,3), datetime.datetime(2024,8,12)))
pixc_read.data

<zcollection.dataset.Dataset>
  Dimensions: ('points: 454911',)
Data variables:
    pass_number    (points)  uint16: dask.array<chunksize=(454911,)>                                                    
    time           (points)  datetime64[ns]: dask.array<chunksize=(454911,)>                                            
    geoid          (points)  float32: dask.array<chunksize=(454911,)>                                                   
    cycle_number   (points)  uint16: dask.array<chunksize=(454911,)>                                                    
    height         (points)  float32: dask.array<chunksize=(454911,)>                                                   
    latitude       (points)  float64: dask.array<chunksize=(454911,)>                                                   
    tile_number    (points)  uint16: dask.array<chunksize=(454911,)>                                                    
    sig0           (points)  float32: dask.array<chunksize=(454911,)>    

In [8]:
gdf_pixc = pixc_read.to_geodataframe()
gdf_pixc

/home/vschaffn/Documents/swot_pixc_study/pixcdust/converters/geo_utils.py:19: UserWarning: No active geometry column to be set. The resulting object will be a pandas.DataFrame with geopandas.GeometryArray(s) containing geometry and CRS information. Use `.set_geometry()` to set an active geometry and upcast to the geopandas.GeoDataFrame manually.
  return ds.xvec.to_geodataframe(*args, **kwargs)


,pass_number,time,geoid,cycle_number,height,latitude,tile_number,sig0,longitude,classification
points,,,,,,,,,,
0,113,2024-08-03 19:45:40,49.860451,19,104.468773,43.400035,231,0.126049,-1.369702,1.0
1,113,2024-08-03 19:45:40,49.850384,19,105.157257,43.401611,231,1.670013,-1.369505,1.0
2,113,2024-08-03 19:45:40,49.850330,19,105.967972,43.401614,231,0.849357,-1.369523,1.0
3,113,2024-08-03 19:45:40,49.848896,19,106.349503,43.401822,231,2.333836,-1.369558,1.0
4,113,2024-08-03 19:45:40,49.850609,19,111.493073,43.400756,231,0.279340,-1.372568,1.0
...,...,...,...,...,...,...,...,...,...,...
454906,113,2024-08-03 19:45:40,49.011883,19,52.735500,43.559886,231,0.310618,-1.481835,1.0
454907,113,2024-08-03 19:45:40,49.011688,19,52.747566,43.559915,231,1.150792,-1.481995,1.0
454908,113,2024-08-03 19:45:40,49.012001,19,51.568275,43.560002,231,0.164267,-1.481359,1.0


Enjoy!